In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp


In [2]:
import sys
sys.path.append(".")
from Transport_class import Transport
from Reaction_class import Reaction

/home/dell/.local/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [3]:
Nx=100
#Ny=10
L=0.045
h=L/Nx
phys_dims=[L]
#phys_dims=[1,1]
#g=pp.CartGrid([Nx,Ny],phys_dims)
g=pp.CartGrid([Nx],phys_dims)
g.compute_geometry()  

In [4]:
unity = np.ones(g.num_cells)
empty = np.empty(0)
porosity=unity
aperture=1

In [5]:
bc_type_Ca=["dir","dir"]
bc_value_Ca=[0.,0.]
#init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_Ca=lambda x,y,z:0.0

In [6]:
bc_type_CaSiO3=["dir","dir"]
bc_value_CaSiO3=[0.,0.]
#init_cond_CaSiO3=lambda x,y,z:int(x<0.5)
init_cond_CaSiO3=lambda x,y,z:16.7504

In [7]:
reaction_data={"temperature":423,
       "A":1.9e-2,
       "rate_const":0.004466,
       "E":4700,
       "R":8.314,
       "K_eq":10e9,
       "ph":3.5,
       "mass_weight":1}

In [8]:
reaction=Reaction(g,reaction_data)
reaction.set_const_rate()

In [9]:
data_reagent_Ca = {"bc_type": bc_type_Ca,
            "bc_value": bc_value_Ca,
            "time_step": 34.56,
            "mass_weight": porosity * aperture,
            "darcy_flux":np.zeros(Nx+1),
            "t_max": 345600,
            "method": "Implicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond_Ca}

In [10]:
data_reagent_CaSiO3= {"bc_type": bc_type_CaSiO3,
            "bc_value": bc_value_CaSiO3,
            "time_step": 34.56,
            "mass_weight": porosity * aperture,
            "darcy_flux":np.zeros(Nx+1),
            "t_max": 345600,
            "method": "Implicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond_CaSiO3}

In [11]:
Ca=np.zeros(g.num_cells)
Ca_transport=Transport(g,data_reagent_Ca)
Ca_lhs,Ca_rhs_b,Ca_rhs_matrix=Ca_transport.set_and_get_matrices(Ca)

In [12]:
CaSiO3=np.zeros(g.num_cells)
CaSiO3_transport=Transport(g,data_reagent_CaSiO3)
CaSiO3_lhs,CaSiO3_rhs_b,CaSiO3_rhs_matrix=CaSiO3_transport.set_and_get_matrices(CaSiO3)

In [13]:
Ca_IEsolver = sps.linalg.factorized(Ca_lhs)
CaSiO3_IEsolver = sps.linalg.factorized(CaSiO3_lhs)

In [14]:
data_transport=Ca_transport.data[pp.PARAMETERS]["transport"]
n_steps = int(np.round(data_transport["t_max"] / data_transport["time_step"]))
dt=data_transport["time_step"]

save_every=1
exporter = pp.Exporter(g, file_name="2_reagents",folder_name="solution")
print(dt)

34.56


In [15]:
for i in range(1,n_steps+1,1):
    
    rhs_react=reaction.compute_rd(Ca,h,Nx)
    if np.isclose(i % save_every, 0):
        # Export existing solution (final export is taken care of below)
        exporter.write_vtu({"Ca":Ca, "CaSiO3":CaSiO3}, time_step=int(i // save_every))
        #exporter.write_vtu({"CaSiO3":CaSiO3},time_step=int(i//save_every))
    if Ca_transport.data[pp.PARAMETERS]["transport"]["method"]=="Explicit":
        Ca = Ca_IEsolver((Ca_rhs_matrix)*Ca+Ca_rhs_b+rhs_react*CaSiO3)
        CaSiO3=CaSiO3_IEsolver((CaSiO3_rhs_matrix)*CaSiO3+CaSiO3_rhs_b-rhs_react*CaSiO3)
    else:
        Ca = Ca_IEsolver((Ca_rhs_matrix)*Ca +Ca_rhs_b+rhs_react*CaSiO3)
        CaSiO3=CaSiO3_IEsolver((CaSiO3_rhs_matrix)*CaSiO3+CaSiO3_rhs_b-rhs_react*CaSiO3)
    
#print(1/dt*A_mass-A_upwind)
print("Ca=")
print(Ca)
print("CaSiO3=")
print(CaSiO3)

exporter.write_vtu({"Ca":Ca, "CaSiO3":CaSiO3}, time_step=(n_steps // save_every))
                         
time_steps = np.arange(
    0, Ca_transport.data[pp.PARAMETERS]["transport"]["t_max"] + dt, save_every * dt
)
print(time_steps.size)
exporter.write_pvd(time_steps)

Ca=
[16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394
 16.7097394 16.7097394 16.7097394 16.7097394 16.7097394 16